## Question 1


In [36]:
!pip install BeautifulSoup4
!pip install requests

In [37]:
#importing libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np


In [38]:
#Linking wiki page and creating soup object
wiki_link = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(wiki_link.text, 'lxml')

#Storing data iteratively using soup object
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
   
    if (index == 0):
        columns = section
    else:
        data.append(section)


df_Toronto = pd.DataFrame(data = data,columns = columns)
df_Toronto.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Removal of Boroughs that are marked as 'Not assigned'


In [39]:
df_Toronto = df_Toronto[df_Toronto['Borough'] != 'Not assigned']
df_Toronto.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods separated with a comma

In [40]:
df_Toronto["Neighbourhood"] = df_Toronto.groupby("Postcode")["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))


df_Toronto = df_Toronto.drop_duplicates()  #Duplicates are removed 


if(df_Toronto.index.name != 'Postcode'):                                  #Index Update
   df_Toronto = df_Toronto.set_index('Postcode')
    
df_Toronto.head()

,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Not assigned


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [41]:
df_Toronto ['Neighbourhood'].replace("Not assigned", df_Toronto ["Borough"],inplace=True)
df_Toronto .head()

,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Queen's Park


### Printing the shape of the dataframe

In [42]:
df_Toronto.shape

(103, 2)

## Question 2

In [43]:
!pip install BeautifulSoup4
!pip install requests

In [47]:
#Get data lat/long data from csv
df_latlong = pd.read_csv("http://cocl.us/Geospatial_data")   

#rename columns and set the index to be Postcode
df_latlong.columns = ["Postcode", "Latitude", "Longitude"]
if(df_latlong.index.name != 'Postcode'):
    df_latlong = df_latlong.set_index('Postcode')
df_latlong.head()

,Latitude,Longitude
Postcode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [48]:
df_Toronto = df_Toronto.join(df_latlong)
df_Toronto.head(11)


,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
M7A,Queen's Park,Queen's Park,43.662301,-79.389494
M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M3B,North York,Don Mills North,43.745906,-79.352188
M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
